In [19]:
require(stringr)
project_name = 'ML-MADNESS'
repo_path = file.path(substr(getwd(),0,gregexpr(pattern = project_name ,
                                                getwd())[[1]][1]-2),
                      project_name)
setwd(repo_path)

In [22]:
source(file.path(repo_path,'objects','helper_functions.R'))
source(file.path(repo_path,'objects','load_directories.R'))
p = scan(file.path(repo_path,'features/requirements.txt'),what="",sep="\n")
load_all_packages(p)
p = scan(file.path(repo_path,'sagemaker/requirements.txt'),what="",sep="\n")
load_all_packages(p)
slice = dplyr::slice
rename = dplyr::rename

Warning message in file(filename, "r", encoding = encoding):
“cannot open file '/home/ec2-user/SageMaker/ML-MADNESS/objects/helper_functions.r': No such file or directory”


ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


[1] "/home/ec2-user/SageMaker/ML-MADNESS"